In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import mne
import random
from mne.decoding import CSP

from sklearn.svm import SVC
from scipy.linalg import fractional_matrix_power
import scipy.signal as signal
import sys
import pickle
import pandas as pd

## 函数定义

定义数据预处理EA, 带通滤波, 采样点划分函数

In [2]:
def ea(x):
    # tensor
    if isinstance(x, np.ndarray):
            x = x.copy()
            x = torch.from_numpy(x).type(torch.FloatTensor)
    for i in range(4):
        s = x[i * 200:(i + 1) * 200]
        cov = torch.zeros((s.shape[0], s.shape[1], s.shape[1]))
        for j in range(s.shape[0]):
            cov[j] = torch.cov(s[j])
        refEA = torch.mean(cov, 0)
        sqrtRefEA = torch.from_numpy(fractional_matrix_power(refEA,-0.5)).type(torch.FloatTensor)
        x[i*200:(i+1)*200] = torch.matmul(sqrtRefEA, x[i*200:(i+1)*200])
    return np.array(x).astype('float64') # 需要修改为float64, 否则ea后加csp报错

def bandpass(x, low=8, high=16):
    b,a = signal.butter(N=4, Wn=(low,high), btype='bandpass', fs=250)
    x_filtered = signal.filtfilt(b,a,x,axis=2)
    return x_filtered

def split_time(x,start=0, end=350):
    return x[:,:,start:end]

定义CSP提取特征

In [3]:
def csp_decode(x, y, csp_num):
    if y.ndim > 1:
        y = y.squeeze()
    csp = CSP(n_components=csp_num, transform_into='average_power')
    x_tran = csp.fit_transform(x, y)

    return csp, x_tran

定义投票函数

In [4]:
def vote(y_pred_ensemble):
    y_pred = np.zeros(y_pred_ensemble.shape[0])
    for i in range(len(y_pred)):
        y_pred[i] = np.argmax(np.bincount(y_pred_ensemble[i]))

    return y_pred

定义获取数据函数

In [5]:
# 获取训练集数据
def get_traindata(root='../data/train/'):
    x1 = []
    y1 = []
    for i in range(4):
        temp = np.load(root + 'S' + str(i + 1) + '.npz')
        x1.append(temp['X'])
        y1.append(temp['y'])
    x = np.array(x1)  #(4, 200, 13, 750)
    y = np.array(y1)  #(4, 200)

    # reshape x: (N,H,W)  y: (N,1) 
    x = x.reshape(x.shape[0] * x.shape[1], x.shape[2], x.shape[3]) # (800, 13, 750)
    y = y.reshape(-1, 1)  # (800, 1) 

    return x,y

# 获取测试集数据
def get_testdata(root='../data/test/'):
    x1 = []
    for i in range(4):
        temp = np.load(root + 'S' + str(i + 5) + '.npz')
        x1.append(temp['X'])
    x = np.array(x1)
    x = x.reshape(x.shape[0]*x.shape[1], x.shape[2],x.shape[3])

    return x

## 模型训练

数据读入, 预处理

In [6]:
x_train_ori,y_train_ori = get_traindata()
x_test_ori = get_testdata()
print(x_train_ori.shape)
print(x_test_ori.shape)

x_train_band = bandpass(x_train_ori, 8,16)
x_test_band = bandpass(x_test_ori, 8,16)

x_train_split = split_time(x_train_band, start=0, end=350)
x_test_split = split_time(x_test_band, start=0, end=350)

x_train_ea = ea(x_train_split)
x_test_ea = ea(x_test_split)

csp, X_train = csp_decode(x_train_ea, y_train_ori, csp_num=5)
X_test = csp.transform(x_test_ea)

(800, 13, 750)
(800, 13, 750)
Computing rank from data with rank=None
    Using tolerance 1.1 (2.2e-16 eps * 13 dim * 3.9e+14  max singular value)
    Estimated rank (mag): 13
    MAG: rank 13 computed from 13 data channels with 0 projectors
Reducing data rank from 13 -> 13
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.2 (2.2e-16 eps * 13 dim * 4.2e+14  max singular value)
    Estimated rank (mag): 13
    MAG: rank 13 computed from 13 data channels with 0 projectors
Reducing data rank from 13 -> 13
Estimating covariance using EMPIRICAL
Done.


In [7]:
# model ensemble
clf_list = []
# 训练新模型
clf = SVC(kernel='rbf')
clf.fit(X_train, y_train_ori.squeeze())
# 第一个是新模型
clf_list.append(clf)

# 加载四个保存模型
for i in range(4):
    load_clf = pickle.load(open('clf'+str(i+1)+'.dat','rb'))
    clf_list.append(load_clf)

print(clf_list)

[SVC(), SVC(), SVC(), SVC(), SVC()]


## 数据预测与保存

In [8]:

y_pred_ensemble = np.zeros((len(y_train_ori),5))
for i in range(5):
    clf = clf_list[i]
    y_pred_ensemble[:,i] = clf.predict(X_test)

print(y_pred_ensemble.shape)

y_pred_ensemble = y_pred_ensemble.astype('int64')
y_pred = vote(y_pred_ensemble).astype('int64')
print(y_pred.shape)

(800, 5)
(800,)


In [9]:
# 数据reshape(200,4), 即每一列是一个被试
new_y_pred = y_pred.reshape(4,200) # reshape 的时候先填充行, 所以一个subject填充一行
new_y_pred = new_y_pred.T
new_y_pred = new_y_pred.astype('int64')

# 查看每个被试的预测情况
print(np.sum(new_y_pred, axis = 0))

# 查看总体预测情况
print(sum(y_pred))

[ 89 120 135 152]
496


In [10]:
# 转成dataframe格式并保存
pd_y_pred = pd.DataFrame(new_y_pred, dtype='int64')
pd_y_pred.columns = ['S5','S6', 'S7', 'S8']
pd_y_pred

,S5,S6,S7,S8
0,1,1,1,1
1,0,1,1,0
2,1,0,1,0
3,1,1,1,0
4,0,0,1,1
...,...,...,...,...
195,0,1,0,1
196,0,0,0,0
197,0,1,1,1
198,1,1,1,1


In [11]:
pd_y_pred.to_csv('y_pred.csv', index=False)

## 查看训练集准确率

In [12]:
# 查看训练集acc
y_train_pred_ensemble = np.zeros((len(y_train_ori),5))
for i in range(5):
    clf = clf_list[i]
    y_train_pred_ensemble[:,i] = clf.predict(X_train)

y_train_pred_ensemble = y_train_pred_ensemble.astype('int64')
y_train_pred = vote(y_train_pred_ensemble)

# 全部使用
print('模型集成后训练集准确率: {:.2f}%'.format(100*np.sum(y_train_pred==y_train_ori.squeeze())/len(y_train_pred)))

# 只使用第一个clf 全部数据训练得到的
print('只使用全部数据训练模型, 训练集准确率: {:.2f}%'.format(100*np.sum(y_train_pred_ensemble[:,0]==y_train_ori.squeeze())/len(y_train_pred)))

模型集成后训练集准确率: 76.62%
只使用全部数据训练模型, 训练集准确率: 76.50%
